<a href="https://colab.research.google.com/github/phil-mira/AMLS_II_assignment24_25/blob/main/AllGradeLevels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model for all Age Groups at School 🏫

This notebook outlines the training and evaluation of the model that has been trained on all of the data. To use this notebook you need to add the "dataset.json", "original_data.json" and "test_question.json" files from the repo to the notebook. Due to the size of the models only A100 GPUs avaliable on Colab can be used for inference/training.

In [ ]:
!pip install datasets
!pip install peft
!pip install trl
!pip install transformers
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation
!pip install accelerate
!pip3 install autoawq

In [ ]:
import json
import os
import torch
import transformers
import gc
from datasets import load_dataset, Dataset, DatasetDict
from peft import LoraConfig, TaskType, get_peft_model, PeftModel, prepare_model_for_kbit_training
from trl import DPOTrainer, DPOConfig
from awq import AutoAWQForCausalLM

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
)

import bitsandbytes
from transformers import BitsAndBytesConfig


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


##Load the preference training dataset into the required format

In [ ]:
# Load dataset
def load_preference_dataset(file_path):
  """
  This function returns the dataset in the required format from the original dataset.

  Args:
    file_path (str): The path to the original dataset.

  Returns:
    dataset_dict (dict): The dataset in the required format.
  """

  with open(file_path, 'r', encoding='utf-8') as f:
      data = json.load(f)

  dataset_dict = {
      "system":[],
      "prompt": [],
      "chosen": [],
      "rejected": []
  }

  for item in data:
      dataset_dict["system"].append("""You are a helpful AI assistant working
                                        in a school setting that provides suitable
                                        answers to a child's questions given their age.""")
      dataset_dict["prompt"].append(item["prompt"][0]["content"])
      dataset_dict["chosen"].append(item["chosen"][0]["content"])
      dataset_dict["rejected"].append(item["rejected"][0]["content"])

  return dataset_dict

## Format the instruction for the model

In [ ]:
def chatml_format(example):
  """
  This function formats the instruction for the model.

  Args:
    example (dict): The example to be formatted.

  Returns:
    dict: The formatted example.
  """

  message = {"role": "system", "content": example['system']}
  system = tokenizer.apply_chat_template([message], tokenize=False)

  # Format instruction
  message = {"role": "user", "content": example['prompt']}

  prompt = tokenizer.apply_chat_template([message], tokenize=False,
                                         add_generation_prompt=True)

  # Format chosen answer
  chosen = example['chosen'] + "<|im_end|>\n"

  # Format rejected answer
  rejected = example['rejected'] + "<|im_end|>\n"

  return {
      "prompt": system + prompt,
      "chosen": chosen,
      "rejected": rejected,
  }


## Set up the data and models - Training Pipeline 🚀


In [ ]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"

new_model = "EducationHermes-2.5-Mistral-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=False)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Load dataset
dataset = load_preference_dataset("dataset.json")

train_size = int(0.9 * len(dataset["prompt"]))

train_dataset = Dataset.from_dict({
    "system": dataset["system"][:train_size],
    "prompt": dataset["prompt"][:train_size],
    "chosen": dataset["chosen"][:train_size],
    "rejected": dataset["rejected"][:train_size]
})

eval_dataset = Dataset.from_dict({
    "system": dataset["system"][train_size:],
    "prompt": dataset["prompt"][train_size:],
    "chosen": dataset["chosen"][train_size:],
    "rejected": dataset["rejected"][train_size:]
})


# Format dataset
train_dataset = train_dataset.map(
    chatml_format,
)

eval_dataset = eval_dataset.map(
    chatml_format,
)

# LoRA Configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
  )

#Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.config.use_cache = False


## Set up the trainers and train the models

In [ ]:
training_args = DPOConfig(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)


# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

# Train the model
dpo_trainer.train()

dpo_trainer.model.save_pretrained("Final")
okenizer.save_pretrained("Final")

import gc
del dpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

# During training the models were uploded to huggingface, these lines of code
# have since been removed for testing purposes


Extracting prompt in train dataset:   0%|          | 0/180 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/180 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/180 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: philippe-miranthis (philippe-miranthis-university-college-london-ucl-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
1,0.693100
2,0.693100
3,0.688100
4,0.692700
5,0.713600
6,0.675800
7,0.667700
8,0.684500
9,0.699800
10,0.631400


TrainOutput(global_step=200, training_loss=0.16913961399564045, metrics={'train_runtime': 2156.4203, 'train_samples_per_second': 1.484, 'train_steps_per_second': 0.093, 'total_flos': 0.0, 'train_loss': 0.16913961399564045, 'epoch': 16.711111111111112})

## Generate Responses for the Trained Model for model evaluation
The results are saved to json file which will then be formatted to a text file for human evaluation. Responses for the base model are also generated for baseline comparison.

In [ ]:
trained_model = "philippe-miranthis/EducationHermes-2.5-Mistral-7B"
base_model = "teknium/OpenHermes-2.5-Mistral-7B"
questions_file = "test_questions.json"

In [ ]:
trained_tokenizer = AutoTokenizer.from_pretrained(trained_model, trust_remote_code=False)
trained_model = AutoModelForCausalLM.from_pretrained(trained_model, trust_remote_code=False)


In [ ]:
with open(questions_file, 'r', encoding='utf-8') as f:
        questions = json.load(f)

results = {"Question": {}, "trained_model": {}, "untrained_model": {}}

# Generate answers for each question
for idx, question in enumerate(questions):

  message = [{"role": "system", "content": """You are a helpful AI assistant
                                                 working in a school setting that
                                                 provides suitable answers to a
                                                 child's questions given their age."""},
            {"role": "user", "content": question}]

  prompt = trained_tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

  # Generate answer with trained model
  pipeline_trained = transformers.pipeline(
      "text-generation",
      model=trained_model,
      tokenizer=trained_tokenizer
  )

  sequences_trained = pipeline_trained(
      prompt,
      do_sample=True,
      temperature=0.7,
      top_p=0.9,
      num_return_sequences=1,
      max_length=200,
  )

  results["trained_model"][f"question_{idx+1}"] = sequences_trained[0]['generated_text']

# Save results to a JSON file
with open("model_comparison_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4)
print("Comparison results saved to model_comparison_results.json")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_i

Comparison results saved to model_comparison_results.json


In [ ]:
del trained_model, trained_tokenizer
gc.collect()
torch.cuda.empty_cache()


NameError: name 'trained_model' is not defined

In [ ]:
untrained_tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=False)
untrained_model = AutoModelForCausalLM.from_pretrained(base_model, trust_remote_code=False)


In [ ]:
with open(questions_file, 'r', encoding='utf-8') as f:
        questions = json.load(f)

results = {"Question": {}, "trained_model": {}, "untrained_model": {}}

for idx, question in enumerate(questions):

  message = [{"role": "system", "content": """You are a helpful AI assistant
                                                 working in a school setting that
                                                 provides suitable answers to a
                                                 child's questions given their age."""},
            {"role": "user", "content": question}]

  prompt = untrained_tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)


  # Generate answer with untrained model or base model
  pipeline_untrained = transformers.pipeline(
      "text-generation",
      model=untrained_model,
      tokenizer=untrained_tokenizer
  )

  sequences_untrained = pipeline_untrained(
      prompt,
      do_sample=True,
      temperature=0.7,
      top_p=0.9,
      num_return_sequences=1,
      max_length=200,
  )

  results["untrained_model"][f"question_{idx+1}"] = sequences_untrained[0]['generated_text']

  results["Question"][f"question_{idx+1}"] = question

  # Save results to a JSON file
with open("model_comparison_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4)
print("Comparison results saved to model_comparison_results.json")

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Device set to use cuda:0
Setting `pad_token_i

Comparison results saved to model_comparison_results.json


## Benchmark the model comparing with base
This section benchmarks the model on standard benchmarks to assess if there is any loss in performance which might hinder the models ability to answer questions correctly. EleutherAI LM_Evaluation harness has been utilized to streamline this process. [Link to their GitHub repo](https://github.com/EleutherAI/lm-evaluation-harness?tab=readme-ov-file)

In [ ]:
!pip install lm-eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 11.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=061a70924316d400fed62ad5fcb9495c1cbec1e2cf0e44a4c588530d800d40ce
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
  Created wheel for sqlitedict: filename=sqlitedict-2

In [ ]:
!lm_eval \
  --model hf \
  --model_args pretrained=philippe-miranthis/EducationHermes-2.5-Mistral-7B \
  --tasks arc_challenge,hellaswag,gsm8k,mmlu_formal_logic,mmlu_high_school_world_history,mmlu_high_school_geography,mmlu_high_school_government_and_politics,mmlu_high_school_biology,mmlu_high_school_chemistry,mmlu_high_school_computer_science,mmlu_high_school_mathematics,mmlu_high_school_physics,sciq \
  --device cuda:0 \
  --batch_size 4

2025-04-27 07:47:12.728735: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 07:47:12.746297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745740032.767647    7914 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745740032.774118    7914 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 07:47:12.795700: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
!lm_eval \
  --model hf \
  --model_args pretrained=teknium/OpenHermes-2.5-Mistral-7B \
  --tasks arc_challenge,hellaswag,gsm8k,mmlu_high_school_formal_logic,
    mmlu_high_school_world_history,mmlu_high_school_geography,
    mmlu_high_school_government_and_politics,mmlu_high_school__biology,
    mmlu_high_school_chemistry,mmlu_high_school_computer_science,
    mmlu_high_school_mathematics,mmlu_high_school_physics,sciq \
  --device cuda:0 \
  --batch_size 4

2025-04-26 12:42:03.393615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745671323.415194    5771 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745671323.421812    5771 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-26:12:42:18,881 INFO     [lm_eval.__main__:379] Selected Tasks: ['arc_challenge', 'gsm8k', 'hellaswag', 'mmlu', 'sciq']
2025-04-26:12:42:18,882 INFO     [lm_eval.evaluator:169] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-04-26:12:42:18,882 INFO     [lm_eval.evaluator:206] Initializing hf model, with arguments: {'pretrained': 'teknium/Open